In [ ]:
import numpy as np
from matplotlib import pylab as plt
%autosave 0
%matplotlib inline

In [1]:
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf 
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


ModuleNotFoundError: No module named 'tensorflow.python.eager'

Vectorize all of the reviews using bag of words
- Keras vectorization implementation: https://keras.io/api/layers/preprocessing_layers/core_preprocessing_layers/text_vectorization/
- Overview of algorithm by Google Cloud Platform: https://www.youtube.com/watch?v=UFtXy0KRxVI

In [ ]:



# creates an object for each item inside of a given tensor, then adds them all into a list
text_dataset = tf.data.Dataset.from_tensor_slices(["foo", "bar", "baz"])
print([x for x in text_dataset])

max_features = 5000  # Maximum vocab size.
max_len = 4  # Sequence length to pad the outputs to.
embedding_dims = 2

# Create the layer.  
vectorize_layer = TextVectorization(
max_tokens=max_features,
output_mode='int',
output_sequence_length=max_len)


print([x.numpy() for x in text_dataset])

# Now that the vocab layer has been created, call `adapt` on the text-only  
# dataset to create the vocabulary. You don't have to batch, but for large  
# datasets this means we're not keeping spare copies of the dataset.  
vectorize_layer.adapt(text_dataset.batch(64))

Run vectorized features through kNN classifier and measure accuracy
- X (features) = vectorized full-reviews
- y (labels) = 'overall' rating